In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from textblob import TextBlob, Word, Blobber
import pandas as pd
import re
Comment = pd.read_csv("/content/drive/My Drive/Colab Notebooks/output.csv", engine='python')
Rev_comm=Comment["content"]

def preprocess(text):
    clean_data = []
    for x in (text[:]): 
        new_text = re.sub('<.*?>', '', x)   # remove HTML tags
        new_text = re.sub(r'[^\w\s]', '', new_text) # remove punc.
        new_text = re.sub(r'\d+','',new_text)# remove numbers
        new_text = new_text.lower() # lower case, .upper() for upper          
        if new_text != '':
            clean_data.append(new_text)
    return clean_data

def tokenization_w(words):
    w_new = []
    for w in (words[:]):
        w_token = word_tokenize(w)
        if w_token != '':
            w_new.append(w_token)
    return w_new

lemmat = WordNetLemmatizer()
def lemmatization(words,wl):
    new = []
    for i in range(wl):
        lem_words = [lemmat.lemmatize(x) for x in (words[:][i])]
        new.append(lem_words)
    return new

clean_comments=preprocess(Rev_comm)
comm_words=tokenization_w(clean_comments)
word_len=len(comm_words)
lem=lemmatization(comm_words,word_len)
print(lem)

new_comment = []
for i in lem:
    new_comment.append(" ".join(i))
print(new_comment)

data = pd.DataFrame({'Comments':new_comment})
print(data['Comments'])

data['word_count'] = data['Comments'].apply(lambda x: len(str(x).split(" ")))
data[['Comments','word_count']].head()

data['char_count'] = data['Comments'].str.len() ## this also includes spaces
data[['Comments','char_count']].head()

def avg_word(sentence):
  words = sentence.split()
  print(words)
  print(len(words))
  print(sum(len(word) for word in words))
  if(len(words)==0):
    ave_w=(sum(len(word) for word in words)/(len(words)+1))
  else:
    ave_w=(sum(len(word) for word in words)/len(words))
  return ave_w

data['avg_word'] = data['Comments'].apply(lambda x: avg_word(x))
data[['Comments','avg_word']].head()

stop = stopwords.words('english')

data['stopwords'] = data['Comments'].apply(lambda x: len([x for x in x.split() if x in stop]))
data[['Comments','stopwords']].head()

data['hastags'] = data['Comments'].apply(lambda x: len([x for x in x.split() if x.startswith('#')]))
data[['Comments','hastags']].head()

data['numerics'] = data['Comments'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))
data[['Comments','numerics']].head()

data['upper'] = data['Comments'].apply(lambda x: len([x for x in x.split() if x.isupper()]))
data[['Comments','upper']].head()

pos_family = {
    'noun' : ['NN','NNS','NNP','NNPS'],
    'pron' : ['PRP','PRP$','WP','WP$'],
    'verb' : ['VB','VBD','VBG','VBN','VBP','VBZ'],
    'adj' :  ['JJ','JJR','JJS'],
    'adv' : ['RB','RBR','RBS','WRB']
}

def check_pos_tag(x, flag):
    cnt = 0
    try:
        wiki = TextBlob(x)
        for tup in wiki.tags:
            ppo = list(tup)[1]
            if ppo in pos_family[flag]:
                cnt += 1
                print(ppo, tup)
    except:
        pass
    return cnt

data['noun_count'] = data['Comments'].apply(lambda x: check_pos_tag(x, 'noun'))
data['verb_count'] = data['Comments'].apply(lambda x: check_pos_tag(x, 'verb'))
data['adj_count'] = data['Comments'].apply(lambda x: check_pos_tag(x, 'adj'))
data['adv_count'] = data['Comments'].apply(lambda x: check_pos_tag(x, 'adv'))
data['pron_count'] = data['Comments'].apply(lambda x: check_pos_tag(x, 'pron'))
data[['Comments','noun_count','verb_count','adj_count', 'adv_count', 'pron_count' ]].head()

data.head()

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

cv=CountVectorizer()
A_vec = cv.fit_transform(new_comment)
print(A_vec.toarray())

tv=TfidfVectorizer()
t_vec = tv.fit_transform(new_comment)
print(t_vec.toarray())

feature_names = tv.get_feature_names()

dense = t_vec.todense()
denselist = dense.tolist()
daf = pd.DataFrame(denselist, columns=feature_names)
print(feature_names)

df_c =pd.concat([daf,data], axis=1)
df_c.head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[['this', 'is', 'very', 'good', 'hardware', 'and', 'speaker', 'however', 'very', 'much', 'handicapped', 'by', 'amazon', 'on', 'software', 'front', 'it', 'can', 'not', 'understand', 'your', 'command', 'percentage', 'time', 'and', 'very', 'limited', 'to', 'amazon', 'music', 'and', 'saavn', 'however', 'you', 'can', 'not', 'use', 'any', 'of', 'good', 'music', 'apps', 'like', 'wynk', 'gaana', 'youtube', 'etc', 'also', 'natural', 'language', 'processing', 'is', 'suppor

,about,accent,access,added,adding,advantage,again,agra,ahead,alexa,all,already,also,am,amazed,amazing,amazon,an,and,answer,any,anyone,apps,april,are,assist,assistant,available,away,bad,base,bass,before,below,better,bit,bulb,but,by,call,...,unfortunately,up,update,use,used,useful,value,very,wa,want,wasool,weird,well,were,what,when,where,which,will,with,worst,would,wynk,yell,you,your,youtube,Comments,word_count,char_count,avg_word,stopwords,hastags,numerics,upper,noun_count,verb_count,adj_count,adv_count,pron_count
0,0.000000,0.000000,0.064322,0.000000,0.000000,0.000000,0.000000,0.064322,0.000000,0.043077,0.000000,0.064322,0.043077,0.000000,0.000000,0.000000,0.207580,0.086155,0.144952,0.064322,0.103790,0.000000,0.128645,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.064322,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.072476,0.051895,0.000000,...,0.000000,0.000000,0.000000,0.064322,0.000000,0.000000,0.000000,0.172310,0.000000,0.000000,0.000000,0.000000,0.000000,0.064322,0.086155,0.043077,0.051895,0.000000,0.051895,0.108714,0.000000,0.064322,0.064322,0.000000,0.217429,0.043077,0.064322,this is very good hardware and speaker however...,173,927,4.364162,87,0,0,0,43,29,16,25,14
1,0.061656,0.000000,0.000000,0.000000,0.000000,0.000000,0.123312,0.000000,0.061656,0.041292,0.049744,0.000000,0.041292,0.000000,0.000000,0.000000,0.099488,0.000000,0.243152,0.000000,0.000000,0.000000,0.000000,0.061656,0.000000,0.061656,0.061656,0.000000,0.061656,0.184968,0.000000,0.000000,0.000000,0.000000,0.123312,0.000000,0.000000,0.104208,0.000000,0.000000,...,0.061656,0.000000,0.123312,0.000000,0.000000,0.000000,0.000000,0.082584,0.000000,0.049744,0.000000,0.000000,0.000000,0.000000,0.082584,0.000000,0.000000,0.061656,0.000000,0.104208,0.123312,0.000000,0.000000,0.061656,0.173680,0.000000,0.000000,it definitely ha better speaker than eco dot g...,169,890,4.272189,71,0,0,0,39,31,22,17,17
2,0.000000,0.078362,0.000000,0.078362,0.078362,0.078362,0.000000,0.000000,0.000000,0.052480,0.000000,0.000000,0.052480,0.156724,0.063222,0.000000,0.000000,0.052480,0.176591,0.000000,0.063222,0.000000,0.000000,0.000000,0.078362,0.000000,0.000000,0.078362,0.000000,0.000000,0.000000,0.078362,0.078362,0.078362,0.000000,0.078362,0.078362,0.088296,0.126444,0.000000,...,0.000000,0.156724,0.000000,0.000000,0.156724,0.078362,0.078362,0.157440,0.156724,0.000000,0.000000,0.078362,0.126444,0.000000,0.000000,0.000000,0.000000,0.000000,0.063222,0.088296,0.000000,0.000000,0.000000,0.000000,0.132444,0.000000,0.000000,this wa launched on th and i got the product b...,155,799,4.161290,73,0,0,0,35,31,20,12,10
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.155435,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.108540,0.000000,0.155435,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.192658,0.000000,0.155435,0.000000,0.000000,0.129025,0.000000,0.000000,0.000000,0.217080,0.000000,0.000000,0.000000,0.000000,0.000000,0.129025,0.000000,it totally paisa wasool speaker is loud she li...,35,185,4.314286,14,0,0,0,8,7,4,4,6
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.218397,0.000000,0.000000,0.000000,0.000000,0.180465,0.000000,0.060430,0.203341,0.000000,0.000000,0.090232,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.145598,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.072799,0.000000,0.000000,0.000000,0.000000,0.060430,0.060430,0.072799,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.152506,0.060430,0.000000,such an amazing product it responds from where...,94,452,3.819149,52,0,0,0,15,